In [1]:
# Dependencies
from sqlalchemy import create_engine
from datetime import timedelta, datetime, tzinfo
from zipfile import ZipFile
import collections
#import psycopg2
import requests
import json
import pandas as pd

In [2]:
##Import the csv data

In [3]:
#data frame for legal status and pop
lgpure_df = pd.read_csv("legaldata.csv")
#data frame for mvr deaths
MVR_df = pd.read_csv("mvdeaths.csv")
#data frame for overdose deaths
overdose_df = pd.read_csv("overdose.csv")

In [4]:
####legal status for map

In [5]:
lgpure_df.head(3)

,State,Pop,legalWeedStatus,medicinalWeedStatus,decriminalizedWeedStatus,state
0,Alabama,4908621,Fully Illegal,False,False,Alabama
1,Alaska,734002,Fully Legal,True,True,Alaska
2,Arizona,7378494,Mixed,True,False,Arizona


In [6]:
lgpure_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   State                     51 non-null     object
 1   Pop                       51 non-null     int64 
 2   legalWeedStatus           51 non-null     object
 3   medicinalWeedStatus       51 non-null     bool  
 4   decriminalizedWeedStatus  51 non-null     bool  
 5   state                     51 non-null     object
dtypes: bool(2), int64(1), object(3)
memory usage: 1.8+ KB


In [7]:
##df to join to other df (MVR and Overdose)
legal_df = lgpure_df.drop(columns=["state"])
legal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   State                     51 non-null     object
 1   Pop                       51 non-null     int64 
 2   legalWeedStatus           51 non-null     object
 3   medicinalWeedStatus       51 non-null     bool  
 4   decriminalizedWeedStatus  51 non-null     bool  
dtypes: bool(2), int64(1), object(2)
memory usage: 1.4+ KB


In [8]:
lgpure_df.head(3)


,State,Pop,legalWeedStatus,medicinalWeedStatus,decriminalizedWeedStatus,state
0,Alabama,4908621,Fully Illegal,False,False,Alabama
1,Alaska,734002,Fully Legal,True,True,Alaska
2,Arizona,7378494,Mixed,True,False,Arizona


In [9]:
###df for legal map


In [11]:
legal_mapdf = lgpure_df.drop(columns=["state","Pop","medicinalWeedStatus","decriminalizedWeedStatus"])
legal_mapdf.head()

,State,legalWeedStatus
0,Alabama,Fully Illegal
1,Alaska,Fully Legal
2,Arizona,Mixed
3,Arkansas,Mixed
4,California,Fully Legal


In [25]:
###export to csv
legal_mapdf.to_csv(r'C:\Users\jcaravella\Documents\Project2\Main_data\state_status.csv', index = False,header=True)

In [11]:
####MVR State Infor

In [12]:
MVR_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 7 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            417 non-null    object 
 1   State                                           416 non-null    object 
 2   Motor Vehicle Deaths - state of residence       416 non-null    float64
 3   Traffic deaths - state of incident              416 non-null    float64
 4   Deaths  per 100,000 population                  416 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       416 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  416 non-null    float64
dtypes: float64(5), object(2)
memory usage: 22.9+ KB


In [12]:
##drop to the same stats as of deaths
years = ['2015','2016','2017']
mvr_neededyears =MVR_df[MVR_df.Year.isin(years)]
mvr_neededyears.head()

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles"
0,2017,U.S. Total,40231.0,37133.0,12.4,1.4,1.2
1,2017,Alabama,1034.0,948.0,21.2,1.9,1.3
2,2017,Alaska,101.0,79.0,13.7,1.0,1.4
3,2017,Arizona,1017.0,1000.0,14.5,1.7,1.5
4,2017,Arkansas,579.0,493.0,19.3,1.7,1.4


In [13]:
##Join legal status
mvr_joined =  mvr_neededyears.join(legal_df.set_index(['State']), on=['State'], how='left')
mvr_joined.info()
#mvr_status = mvr_joined.drop.na()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 0 to 155
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            156 non-null    object 
 1   State                                           156 non-null    object 
 2   Motor Vehicle Deaths - state of residence       156 non-null    float64
 3   Traffic deaths - state of incident              156 non-null    float64
 4   Deaths  per 100,000 population                  156 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       156 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  156 non-null    float64
 7   Pop                                             153 non-null    float64
 8   legalWeedStatus                                 153 non-null    object 
 9   medicinalWeedStatus                        

In [14]:
#drop U.s.total
mvr_status = mvr_joined.dropna()
mvr_status.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            153 non-null    object 
 1   State                                           153 non-null    object 
 2   Motor Vehicle Deaths - state of residence       153 non-null    float64
 3   Traffic deaths - state of incident              153 non-null    float64
 4   Deaths  per 100,000 population                  153 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       153 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  153 non-null    float64
 7   Pop                                             153 non-null    float64
 8   legalWeedStatus                                 153 non-null    object 
 9   medicinalWeedStatus                        

In [15]:
mvr_status.head(3)

,Year,State,Motor Vehicle Deaths - state of residence,Traffic deaths - state of incident,"Deaths per 100,000 population","Traffic deaths per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles",Pop,legalWeedStatus,medicinalWeedStatus,decriminalizedWeedStatus
1,2017,Alabama,1034.0,948.0,21.2,1.9,1.3,4908621.0,Fully Illegal,False,False
2,2017,Alaska,101.0,79.0,13.7,1.0,1.4,734002.0,Fully Legal,True,True
3,2017,Arizona,1017.0,1000.0,14.5,1.7,1.5,7378494.0,Mixed,True,False


In [17]:
####Mvr state level table
#drop columns I don't need for either state level tabel graph

mvr_state = mvr_status.drop(columns=["Motor Vehicle Deaths - state of residence","Traffic deaths - state of incident","Traffic deaths - state of incident","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","Pop","decriminalizedWeedStatus","medicinalWeedStatus"])
                   
mvr_state.head()                                       


,Year,State,"Deaths per 100,000 population",legalWeedStatus
1,2017,Alabama,21.2,Fully Illegal
2,2017,Alaska,13.7,Fully Legal
3,2017,Arizona,14.5,Mixed
4,2017,Arkansas,19.3,Mixed
5,2017,California,10.6,Fully Legal


In [18]:
mvr_state.head(3)

,Year,State,"Deaths per 100,000 population",legalWeedStatus
1,2017,Alabama,21.2,Fully Illegal
2,2017,Alaska,13.7,Fully Legal
3,2017,Arizona,14.5,Mixed


In [19]:
##MVR deaths by legal status

In [18]:
mvr_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Year                                            153 non-null    object 
 1   State                                           153 non-null    object 
 2   Motor Vehicle Deaths - state of residence       153 non-null    float64
 3   Traffic deaths - state of incident              153 non-null    float64
 4   Deaths  per 100,000 population                  153 non-null    float64
 5   Traffic deaths  per 10,000 motor vehicles       153 non-null    float64
 6   Traffic deaths - per 100,000,000 vehicle miles  153 non-null    float64
 7   Pop                                             153 non-null    float64
 8   legalWeedStatus                                 153 non-null    object 
 9   medicinalWeedStatus                        

In [19]:
###drop columns I don't need

data_legal = mvr_status.drop(columns=["State","Motor Vehicle Deaths - state of residence","Deaths  per 100,000 population","Traffic deaths  per 10,000 motor vehicles","Traffic deaths - per 100,000,000 vehicle miles","medicinalWeedStatus","decriminalizedWeedStatus"])
#mvr_bystatus = totaldeaths.groupby(['State','Year','Indicator','State Name'])['Data Value'].sum().reset_index()
data_legal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Year                                153 non-null    object 
 1   Traffic deaths - state of incident  153 non-null    float64
 2   Pop                                 153 non-null    float64
 3   legalWeedStatus                     153 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.0+ KB


In [20]:
#sums by legal status 
#pop sum
status_pop = data_legal.groupby(['Year','legalWeedStatus'])['Pop'].sum().reset_index()
status_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Pop              9 non-null      float64
dtypes: float64(1), object(2)
memory usage: 344.0+ bytes


In [21]:
status_pop.head()

,Year,legalWeedStatus,Pop
0,2015,Fully Illegal,49127995.0
1,2015,Fully Legal,81471023.0
2,2015,Mixed,200719974.0
3,2016,Fully Illegal,49127995.0
4,2016,Fully Legal,81471023.0


In [23]:
##MVR Sum
data_legal.head()

,Year,Traffic deaths - state of incident,Pop,legalWeedStatus
1,2017,948.0,4908621.0,Fully Illegal
2,2017,79.0,734002.0,Fully Legal
3,2017,1000.0,7378494.0,Mixed
4,2017,493.0,3038999.0,Mixed
5,2017,3602.0,39937489.0,Fully Legal


In [25]:
#rename column name
d2 = data_legal.rename(columns={"Traffic deaths - state of incident":"Traffic_deaths"})

d2.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 1 to 155
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             153 non-null    object 
 1   Traffic_deaths   153 non-null    float64
 2   Pop              153 non-null    float64
 3   legalWeedStatus  153 non-null    object 
dtypes: float64(2), object(2)
memory usage: 6.0+ KB


In [26]:
#MVR sums by legal status

status_mvrtotal = d2.groupby(['Year','legalWeedStatus'])['Traffic_deaths'].sum().reset_index()
status_mvrtotal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Traffic_deaths   9 non-null      float64
dtypes: float64(1), object(2)
memory usage: 344.0+ bytes


In [27]:
status_mvrtotal.head()

,Year,legalWeedStatus,Traffic_deaths
0,2015,Fully Illegal,7262.0
1,2015,Fully Legal,6632.0
2,2015,Mixed,21198.0
3,2016,Fully Illegal,8131.0
4,2016,Fully Legal,7752.0


In [28]:
#first join for by status table

In [29]:
status_mvrtotal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Traffic_deaths   9 non-null      float64
dtypes: float64(1), object(2)
memory usage: 344.0+ bytes


In [30]:
status_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Pop              9 non-null      float64
dtypes: float64(1), object(2)
memory usage: 344.0+ bytes


In [31]:
###join all the status info into one table
status_join1 = status_pop.join(status_mvrtotal.set_index(['legalWeedStatus','Year']),on=['legalWeedStatus','Year'],how='left')
status_join1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Pop              9 non-null      float64
 3   Traffic_deaths   9 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [32]:
###Overdose Info###

In [33]:
overdose_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29568 entries, 0 to 29567
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   State                          29568 non-null  object 
 1   Year                           29568 non-null  int64  
 2   Month                          29568 non-null  object 
 3   Period                         29568 non-null  object 
 4   Indicator                      29568 non-null  object 
 5   Data Value                     24236 non-null  float64
 6   Percent Complete               29568 non-null  object 
 7   Percent Pending Investigation  29568 non-null  float64
 8   State Name                     29568 non-null  object 
 9   Footnote                       29568 non-null  object 
 10  Footnote Symbol                29568 non-null  object 
 11  Predicted Value                18465 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usag

In [34]:
###filter clean to only columns I need

##od to the same years as mvr
overdose = overdose_df[overdose_df.Year.isin(years)]
#states, indicator, year
od_drop_df = overdose.drop(columns=["Period","Percent Complete","Percent Pending Investigation","Footnote","Footnote Symbol","Footnote Symbol","Predicted Value"])
###join the status
od_df = od_drop_df.join(legal_mapdf.set_index(['State']), on=['State Name'],how='left')
od_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19008 entries, 0 to 29327
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            19008 non-null  object 
 1   Year             19008 non-null  int64  
 2   Month            19008 non-null  object 
 3   Indicator        19008 non-null  object 
 4   Data Value       14566 non-null  float64
 5   State Name       19008 non-null  object 
 6   legalWeedStatus  18144 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ MB


In [35]:
###Total deaths

In [36]:
#filter to the number of deaths

totaldeaths1 = od_df[od_df['Indicator']=='Number of Deaths']
totaldeaths =  totaldeaths1.drop(columns=['Indicator'])

totaldeaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908 entries, 4 to 29321
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            1908 non-null   object 
 1   Year             1908 non-null   int64  
 2   Month            1908 non-null   object 
 3   Data Value       1908 non-null   float64
 4   State Name       1908 non-null   object 
 5   legalWeedStatus  1836 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 104.3+ KB


In [37]:
##sum up deaths for state
state_yr_deaths1 = totaldeaths.groupby(['State','Year','State Name','legalWeedStatus'])['Data Value'].sum().reset_index()
state_yr_deaths = state_yr_deaths1.rename(columns={"Data Value":"yearly_deaths"})

state_yr_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            153 non-null    object 
 1   Year             153 non-null    int64  
 2   State Name       153 non-null    object 
 3   legalWeedStatus  153 non-null    object 
 4   yearly_deaths    153 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 6.1+ KB


In [38]:
state_yr_deaths.head()



,State,Year,State Name,legalWeedStatus,yearly_deaths
0,AK,2015,Alaska,Fully Legal,50086.0
1,AK,2016,Alaska,Fully Legal,51141.0
2,AK,2017,Alaska,Fully Legal,51597.0
3,AL,2015,Alabama,Fully Illegal,607641.0
4,AL,2016,Alabama,Fully Illegal,608611.0


In [39]:
####deaths by legal status

In [45]:
status_deaths = state_yr_deaths.groupby(['Year','legalWeedStatus'])['yearly_deaths'].sum().reset_index()
status_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      int64  
 1   legalWeedStatus  9 non-null      object 
 2   yearly_deaths    9 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [46]:
status_deaths.head()

,Year,legalWeedStatus,yearly_deaths
0,2015,Fully Illegal,5410542.0
1,2015,Fully Legal,7034178.0
2,2015,Mixed,19335816.0
3,2016,Fully Illegal,5441650.0
4,2016,Fully Legal,7132055.0


In [ ]:
####overdose totals

In [41]:
totalod = od_df[od_df['Indicator']=='Number of Drug Overdose Deaths']
totalod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1908 entries, 0 to 29322
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            1908 non-null   object 
 1   Year             1908 non-null   int64  
 2   Month            1908 non-null   object 
 3   Indicator        1908 non-null   object 
 4   Data Value       1908 non-null   float64
 5   State Name       1908 non-null   object 
 6   legalWeedStatus  1836 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 119.2+ KB


In [51]:
state_yr_od1 = totalod.groupby(['Year','legalWeedStatus','State Name'])['Data Value'].sum().reset_index()
state_yr_od = state_yr_od1.rename(columns={"Data Value":"yearly_OD_deaths"})
state_yr_od.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              153 non-null    int64  
 1   legalWeedStatus   153 non-null    object 
 2   State Name        153 non-null    object 
 3   yearly_OD_deaths  153 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.9+ KB


In [52]:
state_yr_od.head()

,Year,legalWeedStatus,State Name,yearly_OD_deaths
0,2015,Fully Illegal,Alabama,8818.0
1,2015,Fully Illegal,Idaho,2570.0
2,2015,Fully Illegal,Kansas,3926.0
3,2015,Fully Illegal,Kentucky,14286.0
4,2015,Fully Illegal,Mississippi,3907.0


In [ ]:
#####od's by legal status####

In [54]:
#drop state
#legal_deaths = state_yr_deaths.drop(columns=["State","State Name"])
####sum by legal type
OD_by_legal_type = state_yr_od.groupby(['Year','legalWeedStatus'])['yearly_OD_deaths'].sum().reset_index()
OD_by_legal_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      int64  
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [57]:
#joins the two together into one
Od_deaths_status_combined = OD_by_legal_type.join(status_deaths.set_index(['Year','legalWeedStatus']), on = ['Year','legalWeedStatus'], how='left')
Od_deaths_status_combined.info()                                                 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      int64  
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
 3   yearly_deaths     9 non-null      float64
dtypes: float64(2), int64(1), object(1)
memory usage: 416.0+ bytes


In [58]:
Od_deaths_status_combined['Year'] = Od_deaths_status_combined['Year'].astype(str)

In [59]:
Od_deaths_status_combined.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      object 
 1   legalWeedStatus   9 non-null      object 
 2   yearly_OD_deaths  9 non-null      float64
 3   yearly_deaths     9 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [66]:
od_per_death = Od_deaths_status_combined['yearly_OD_deaths']/Od_deaths_status_combined['yearly_deaths']
od_per_death

0    0.016533
1    0.020075
2    0.018626
3    0.018316
4    0.021609
5    0.022112
6    0.020442
7    0.022897
8    0.026359
dtype: float64

In [ ]:
###by status table

In [45]:
status_join1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             9 non-null      object 
 1   legalWeedStatus  9 non-null      object 
 2   Pop              9 non-null      float64
 3   Traffic_deaths   9 non-null      float64
dtypes: float64(2), object(2)
memory usage: 416.0+ bytes


In [60]:
total_joined_status = status_join1.join(Od_deaths_status_combined.set_index(['Year','legalWeedStatus']), on = ['Year','legalWeedStatus'], how='left')
total_joined_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              9 non-null      object 
 1   legalWeedStatus   9 non-null      object 
 2   Pop               9 non-null      float64
 3   Traffic_deaths    9 non-null      float64
 4   yearly_OD_deaths  9 non-null      float64
 5   yearly_deaths     9 non-null      float64
dtypes: float64(4), object(2)
memory usage: 560.0+ bytes


In [62]:
OD_type_status = [['Year', total_joined_status.Year]]
OD_type_status.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
##into postgres

In [17]:
#rds_connection_string = f"{dbuser}<insert user name>:<insert password>@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')
password = "Hooch#12"
dbuser = "postgres"
rds_connection_string = f"{dbuser}:{password}@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
###postgresSQL
##push to server

In [ ]:
engine.table_names()

In [ ]:
#pushing Cluster,Bar,Line DATAFRAME to SQL
years_trend.to_sql(name='guns_year', con=engine, if_exists='replace', index=False)
deaths_2018_unindexed.to_sql(name='killings_injuries_2018', con=engine, if_exists='replace', index=False)

In [ ]:

#Creating CSV's of Cluster,Bar,Line DATAFRAME
years_trend.to_csv('year_trend_gun.csv')
deaths_2018.to_csv('deaths_2018.csv')
gun14_monthly.to_csv("gun2014onwrd.csv")